In [1]:
import os, gc, sys, time, random, math

import torch
import torch.nn as nn
import torch.nn.functional as F

from typing import Optional, List

from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from diffusers import StableDiffusionPipeline
from tqdm import tqdm

from PIL import Image

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


In [2]:
class ImageDataset(Dataset):
    def __init__(self, root_dir, df, size = 224, center_crop = True):
        self.root_dir = root_dir
        self.files = df['file_name'].tolist()
        self.findings = df['text'].tolist()
        # self.tokenizer = tokenizer
        self.image_transforms = transforms.ToTensor()
        # transforms.Compose(
        #     [
        #         transforms.Resize(size, interpolation=transforms.InterpolationMode.BILINEAR),
        #         transforms.CenterCrop(size) if center_crop else transforms.RandomCrop(size),
        #         transforms.ToTensor(),
        #         transforms.Normalize([0.5], [0.5]),
        #     ]
        # )


    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        example = {}
        instance_image = Image.open(
            os.path.join(self.root_dir, self.files[idx])
        ).convert("RGB")

        example["instance_images"] = self.image_transforms(instance_image)
        example["instance_prompt_ids"] = self.findings[idx]
        # self.tokenizer(
        #     self.findings[idx],
        #     truncation=True,
        #     padding="max_length",
        #     max_length=self.tokenizer.model_max_length,
        #     return_tensors="pt",
        # ).input_ids

        return example

In [3]:
# The main path
Main_Path = '/home/mcrespo/migros_deepL'
# The datasets path under the main path
Data_storage = Main_Path + '/sample_flair'
save_result_path = Main_Path + '/selora_outputs'
reports_path = Data_storage + '/metadata.csv'
### folder to save the result.
folder_name = 'loras'

metadata = pd.read_csv(reports_path)
train_df, temp_df = train_test_split(metadata, test_size=0.2, random_state=42)
valid_df, test_df = train_test_split(temp_df, test_size=0.2, random_state=42)

train_ds = ImageDataset(
    root_dir=Data_storage,
    df=train_df
)

train_loader = DataLoader(train_ds, batch_size=2, shuffle=True, num_workers = 0)
### batch size determines the number of steps for each epoch, we are doing 100 epochs. So total number of steps is : 100 * train_df//batchsize

In [15]:
from safetensors.torch import load_file
model_unet = '/work/scratch/mcrespo/output/test_12_26_alldataset/12-27_16h45m08s/selora_outputs/loras/trained_model/final_Unet/diffusion_pytorch_model.safetensors'
model_text= '/work/scratch/mcrespo/output/test_12_26_alldataset/12-27_16h45m08s/selora_outputs/loras/trained_model/final_Text/model.safetensors'

model_text = load_file(model_text)
model_unet = load_file(model_unet)

In [20]:
for tensor_name in model_text.keys():
    print(tensor_name)

text_model.embeddings.position_embedding.weight
text_model.embeddings.token_embedding.weight
text_model.encoder.layers.0.layer_norm1.bias
text_model.encoder.layers.0.layer_norm1.weight
text_model.encoder.layers.0.layer_norm2.bias
text_model.encoder.layers.0.layer_norm2.weight
text_model.encoder.layers.0.mlp.fc1.bias
text_model.encoder.layers.0.mlp.fc1.lora_A
text_model.encoder.layers.0.mlp.fc1.lora_A_temp
text_model.encoder.layers.0.mlp.fc1.lora_B
text_model.encoder.layers.0.mlp.fc1.lora_B_temp
text_model.encoder.layers.0.mlp.fc1.weight
text_model.encoder.layers.0.mlp.fc2.bias
text_model.encoder.layers.0.mlp.fc2.lora_A
text_model.encoder.layers.0.mlp.fc2.lora_A_temp
text_model.encoder.layers.0.mlp.fc2.lora_B
text_model.encoder.layers.0.mlp.fc2.lora_B_temp
text_model.encoder.layers.0.mlp.fc2.weight
text_model.encoder.layers.0.self_attn.k_proj.bias
text_model.encoder.layers.0.self_attn.k_proj.lora_A
text_model.encoder.layers.0.self_attn.k_proj.lora_A_temp
text_model.encoder.layers.0.self

In [16]:
for name, layer in model_text.named_modules():
    print(name)

AttributeError: 'dict' object has no attribute 'named_modules'